In [2]:
import cv2
import matplotlib.pyplot as plt
import os
from ultralytics import YOLO
import pandas as pd
import numpy as np

model = YOLO('yolov8n-pose.pt')

directory_path = '/home/mind/projects/projects/action_recognition/data'

df = pd.DataFrame(columns=['Nose_x', 'Nose_y', 'Left-eye_x', 'Left-eye_y', 'Right-eye_x', 'Right-eye_y', 'Left-ear_x', 'Left-ear_y','Right-ear_x', 'Right-ear_y', 'Left-shoulder_x', 'Left-shoulder_y', 'Right-shoulder_x', 'Right-shoulder_y', 'Left-elbow_x', 'Left-elbow_y', 'Right-elbow_x', 'Right-elbow_y', 'Left-wrist_x', 'Left-wrist_y', 'Right-wrist_x', 'Right-wrist_y', 'Left-hip_x', 'Left-hip_y', 'Right-hip_x', 'Right-hip_y', 'Left-knee_x', 'Left-knee_y', 'Right-knee_x', 'Right-knee_y', 'Left-ankle_x', 'Left-ankle_y', 'Right-ankle_x', 'Right-ankle_y', 'Class'])

for sub_directory in os.listdir(directory_path):

    sub_directory_path = os.path.join(directory_path, sub_directory)
    if sub_directory.lower() == 'standing':
        # class_encoding = np.array([1,0,0])
        class_encoding = 0
    elif sub_directory.lower() == 'falling':
        # class_encoding = np.array([0,1,0])
        class_encoding = 1
    elif sub_directory.lower() == 'walking':
        # class_encoding = np.array([0,0,1])
        class_encoding = 2

    for image in os.listdir(sub_directory_path):
        image_path = os.path.join(sub_directory_path, image)
        # img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        img = cv2.imread(image_path)
        # plt.imshow(img)
        # plt.show()
        results = model(source=img,
                show=False,
                conf=0.3,
                save=False,
                verbose=False,
                stream=True,
        )
        for result in results:
            keypoints = result.keypoints.xyn[0].numpy()
            df.loc[len(df.index)] = [keypoints[0][0],
                                     keypoints[0][1],
                                     keypoints[1][0],
                                     keypoints[1][1],
                                     keypoints[2][0],
                                     keypoints[2][1],
                                     keypoints[3][0],
                                     keypoints[3][1],
                                     keypoints[4][0],
                                     keypoints[4][1],
                                     keypoints[5][0],
                                     keypoints[5][1],
                                     keypoints[6][0],
                                     keypoints[6][1],
                                     keypoints[7][0],
                                     keypoints[7][1],
                                     keypoints[8][0],
                                     keypoints[8][1],
                                     keypoints[9][0],
                                     keypoints[9][1],
                                     keypoints[10][0],
                                     keypoints[10][1],
                                     keypoints[11][0],
                                     keypoints[11][1],
                                     keypoints[12][0],
                                     keypoints[12][1],
                                     keypoints[13][0],
                                     keypoints[13][1],
                                     keypoints[14][0],
                                     keypoints[14][1],
                                     keypoints[15][0],
                                     keypoints[15][1],
                                     keypoints[16][0],
                                     keypoints[16][1],
                                     class_encoding]  
   
    print('----------------------------------------------------------------------------------')
df

----------------------------------------------------------------------------------
----------------------------------------------------------------------------------
----------------------------------------------------------------------------------


,Nose_x,Nose_y,Left-eye_x,Left-eye_y,Right-eye_x,Right-eye_y,Left-ear_x,Left-ear_y,Right-ear_x,Right-ear_y,...,Right-hip_y,Left-knee_x,Left-knee_y,Right-knee_x,Right-knee_y,Left-ankle_x,Left-ankle_y,Right-ankle_x,Right-ankle_y,Class
0,0.478284,0.133413,0.499803,0.116675,0.473419,0.119399,0.568405,0.124873,0.000000,0.000000,...,0.464843,0.577666,0.661005,0.563175,0.654282,0.614005,0.847194,0.593564,0.838853,0.0
1,0.454423,0.141408,0.477490,0.127787,0.434566,0.124023,0.000000,0.000000,0.377019,0.111535,...,0.439932,0.331944,0.656935,0.247747,0.670391,0.291378,0.877636,0.263693,0.896337,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.578014,0.153296,0.000000,0.000000,...,0.511131,0.554743,0.722128,0.722117,0.713977,0.626706,0.919435,0.753418,0.908243,0.0
3,0.198645,0.160323,0.201286,0.153451,0.192050,0.152196,0.000000,0.000000,0.174223,0.154628,...,0.329621,0.224454,0.424685,0.184765,0.432142,0.223020,0.519999,0.179691,0.533525,0.0
4,0.651253,0.085236,0.663306,0.070630,0.639246,0.069483,0.678602,0.083491,0.620734,0.081729,...,0.441787,0.678815,0.655587,0.617253,0.645899,0.667261,0.841608,0.636943,0.829176,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,0.380328,0.164434,0.384123,0.152690,0.369777,0.151617,0.000000,0.000000,0.338227,0.161198,...,0.526652,0.286939,0.714624,0.398870,0.718725,0.238173,0.905916,0.434722,0.913952,2.0
191,0.481028,0.126194,0.000000,0.000000,0.462289,0.111885,0.000000,0.000000,0.380191,0.123573,...,0.493082,0.281861,0.700545,0.460273,0.690499,0.201148,0.906666,0.499733,0.890266,2.0
192,0.334331,0.300614,0.000000,0.000000,0.332278,0.296055,0.000000,0.000000,0.323723,0.300239,...,0.438780,0.287157,0.500909,0.352143,0.511288,0.248724,0.557779,0.355508,0.575060,2.0
193,0.526324,0.323159,0.535386,0.314356,0.517173,0.310068,0.000000,0.000000,0.497830,0.310550,...,0.591332,0.467159,0.730629,0.531427,0.744855,0.395297,0.839091,0.529664,0.882046,2.0


In [3]:
df = df.sample(frac = 1, ignore_index=True)
df

,Nose_x,Nose_y,Left-eye_x,Left-eye_y,Right-eye_x,Right-eye_y,Left-ear_x,Left-ear_y,Right-ear_x,Right-ear_y,...,Right-hip_y,Left-knee_x,Left-knee_y,Right-knee_x,Right-knee_y,Left-ankle_x,Left-ankle_y,Right-ankle_x,Right-ankle_y,Class
0,0.448218,0.214142,0.443878,0.192227,0.000000,0.000000,0.465537,0.154066,0.000000,0.000000,...,0.329305,0.632770,0.504261,0.476094,0.463430,0.710204,0.689874,0.391260,0.616979,1.0
1,0.796002,0.774082,0.000000,0.000000,0.796246,0.751672,0.000000,0.000000,0.749214,0.704806,...,0.635519,0.622788,0.490075,0.589294,0.536125,0.543509,0.434142,0.558632,0.501543,1.0
2,0.590899,0.439875,0.607431,0.432453,0.000000,0.000000,0.652161,0.442893,0.000000,0.000000,...,0.663147,0.665015,0.793273,0.652343,0.789115,0.680835,0.910118,0.669661,0.906476,0.0
3,0.335276,0.277567,0.338181,0.269241,0.327588,0.265546,0.000000,0.000000,0.300299,0.266984,...,0.559722,0.268787,0.675941,0.368869,0.696811,0.207831,0.798830,0.400733,0.830538,2.0
4,0.374782,0.589757,0.360380,0.579657,0.364018,0.613508,0.000000,0.000000,0.372785,0.670286,...,0.683231,0.634260,0.457704,0.649277,0.554626,0.752903,0.515517,0.751523,0.602514,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.309816,0.668182,0.465963,0.689535,...,0.402217,0.392482,0.270476,0.581533,0.280438,0.438516,0.154955,0.595174,0.140100,1.0
191,0.422915,0.125921,0.438496,0.108651,0.000000,0.000000,0.509244,0.113394,0.000000,0.000000,...,0.514496,0.773582,0.739471,0.302404,0.691970,0.947168,0.946721,0.203496,0.881561,2.0
192,0.718161,0.216540,0.736782,0.209151,0.718458,0.196067,0.758909,0.222229,0.000000,0.000000,...,0.453266,0.652408,0.638070,0.516609,0.667181,0.654284,0.696066,0.451517,0.787326,1.0
193,0.544150,0.253655,0.568572,0.247520,0.522286,0.241296,0.600739,0.269832,0.494460,0.258898,...,0.482476,0.647020,0.471678,0.331335,0.453705,0.560877,0.526196,0.217802,0.514018,1.0


In [4]:
x_train = df.drop(columns='Class')
y_train = df['Class']
print(x_train, y_train)

# x_train = df.iloc[:, :17].values
# y_train = df.iloc[:, -1].values
# print(x_train, y_train)

       Nose_x    Nose_y  Left-eye_x  Left-eye_y  Right-eye_x  Right-eye_y  \
0    0.448218  0.214142    0.443878    0.192227     0.000000     0.000000   
1    0.796002  0.774082    0.000000    0.000000     0.796246     0.751672   
2    0.590899  0.439875    0.607431    0.432453     0.000000     0.000000   
3    0.335276  0.277567    0.338181    0.269241     0.327588     0.265546   
4    0.374782  0.589757    0.360380    0.579657     0.364018     0.613508   
..        ...       ...         ...         ...          ...          ...   
190  0.000000  0.000000    0.000000    0.000000     0.000000     0.000000   
191  0.422915  0.125921    0.438496    0.108651     0.000000     0.000000   
192  0.718161  0.216540    0.736782    0.209151     0.718458     0.196067   
193  0.544150  0.253655    0.568572    0.247520     0.522286     0.241296   
194  0.000000  0.000000    0.000000    0.000000     0.000000     0.000000   

     Left-ear_x  Left-ear_y  Right-ear_x  Right-ear_y  ...  Right-hip_x  \


In [ ]:
print(x_train.iloc[0, :])

In [5]:
from sklearn.svm import SVC

In [27]:
svc = SVC(C=0.5)
clf = svc.fit(x_train, y_train)

In [28]:
# outputs = model(source='test_data/standing0.jpg',
#                 show=False,
#                 conf=0.3,
#                 save=False,
#                 verbose=False,
#                 stream=True,
#         )

# for output in outputs:
#     output_keypoints = output.keypoints.xyn.numpy()
#     # print(output_keypoints)

# prediction = clf.predict(output_keypoints.flatten().reshape(1, -1))

In [29]:
# print(prediction[0])

In [35]:
cap = cv2.VideoCapture('/home/mind/projects/projects/action_recognition/test_data/pexels-kampus-production-8711342 (720p).mp4')

if (cap.isOpened() == False):
    print('Error while trying to read video. Please check path again')

while(cap.isOpened):

    ret, frame = cap.read()

    if ret:

        outputs = model(source=frame,
                show=False,
                conf=0.3,
                save=False,
                verbose=False,
                stream=True,
        )

        for output in outputs:
            output_keypoints = output.keypoints.xyn.numpy()
        
        # print(len(output_keypoints[0]))
        
        text = ''
        flag = False

        if len(output_keypoints[0]) == 17:
            prediction = clf.predict(output_keypoints[0].flatten().reshape(1, -1))
            flag = True
            # print(prediction[0])
        # else:
        #     text = ''

        if flag:
            if prediction[0] == float(0):
                text = 'standing'
            elif prediction[0] == float(1):
                text = 'falling'
            elif prediction[0] == float(2):
                text = 'walking'

        frame = cv2.putText(frame.copy(), text, (425, 100), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1.5, color=[0, 0, 255], thickness=2, lineType=cv2.LINE_AA)

        cv2.imshow('output', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'): 
                break
    else:
        break
    
cap.release()
cv2.destroyAllWindows() 

/home/mind/projects/virtual_environments/pose_estimation_venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/home/mind/projects/virtual_environments/pose_estimation_venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/home/mind/projects/virtual_environments/pose_estimation_venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/home/mind/projects/virtual_environments/pose_estimation_venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/home/mind/projects/virtual_environments/pose_estimation_venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does 